pre trained models


In [ ]:
import os
if not os.path.exists("/"):
  os.makedirs("/content/drive/MyDrive/sovits4data/logs/44k")

!wget https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/D_0.pth -P /content/drive/MyDrive/sovits4data/logs/44k
!wget https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/G_0.pth -P /content/drive/MyDrive/sovits4data/logs/44k

cloning repo and installing dependencies

In [ ]:
!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.0
!pip uninstall -y torchdata torchtext
!pip install --upgrade pip setuptools numpy numba
!pip install pyworld praat-parselmouth fairseq tensorboardX torchcrepe librosa==0.9.1
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

%cd /content/so-vits-svc

!curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o /content/so-vits-svc/nsf_hifigan_20221211.zip
!unzip nsf_hifigan_20221211.zip
!rm -rf pretrain/nsf_hifigan
!mv -v nsf_hifigan pretrain

!curl -L https://ibm.ent.box.com/shared/static/z1wgl1stco8ffooyatzdwsqn2psd9lrr -o /content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt

writing files and paths

In [ ]:
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"  

RAW_DIR = sovits_data_dir + "raw/"
RESULTS_DIR = sovits_data_dir + "results/"
FILELISTS_DIR = sovits_data_dir + "filelists/"
CONFIGS_DIR = sovits_data_dir + "configs/"
LOGS_DIR = sovits_data_dir + "logs/44k/"
config_path = "configs/config.json"
sync_raw_and_results = True 
if sync_raw_and_results:
  !mkdir -p {RAW_DIR}
  !mkdir -p {RESULTS_DIR}
  !rm -rf /content/so-vits-svc/raw
  !rm -rf /content/so-vits-svc/results
  !ln -s {RAW_DIR} /content/so-vits-svc/raw
  !ln -s {RESULTS_DIR} /content/so-vits-svc/results


sync_configs_and_logs = True  
if sync_configs_and_logs:
  !mkdir -p {FILELISTS_DIR}
  !mkdir -p {CONFIGS_DIR}
  !mkdir -p {LOGS_DIR}
  !rm -rf /content/so-vits-svc/filelists
  !rm -rf /content/so-vits-svc/configs
  !rm -rf /content/so-vits-svc/logs/44k
  !ln -s {FILELISTS_DIR} /content/so-vits-svc/filelists
  !ln -s {CONFIGS_DIR} /content/so-vits-svc/configs
  !ln -s {LOGS_DIR} /content/so-vits-svc/logs/44k

link dataset

In [ ]:
# link dataset in dataset_raw
!ln -s /content/drive/MyDrive/sliced/ /content/so-vits-svc/dataset_raw/

resample


In [ ]:
%cd /content/so-vits-svc
!python resample.py

generate config


In [ ]:

%cd /content/so-vits-svc

# speech_encoder = "vec768l12"

# from pretrain.meta import download_dict
# download_dict = download_dict()

# url = download_dict[speech_encoder]["url"]
# output = download_dict[speech_encoder]["output"]

# import os
# if not os.path.exists(output):
#   !curl -L {url} -o {output}

!python preprocess_flist_config.py #--speech_encoder={speech_encoder}

generate hubert


In [ ]:
%cd /content/so-vits-svc
!python preprocess_hubert_f0.py

saving

In [ ]:
sovits_data_dir = "/content/drive/MyDrive/sovits4data/" 

!mkdir -p {sovits_data_dir}
!zip -r dataset.zip /content/so-vits-svc/dataset
!cp -vr dataset.zip "{sovits_data_dir}"